In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/game-of-thrones-books/A Storm Of Swords(P3).txt
/kaggle/input/game-of-thrones-books/A Feast For Crows(P4).txt
/kaggle/input/game-of-thrones-books/A Game Of Thrones(P1).txt
/kaggle/input/game-of-thrones-books/A Dance With Dragons(P5).txt
/kaggle/input/game-of-thrones-books/A Clash of Kings(P2).txt


In [2]:
import pandas as pd
data = pd.DataFrame(
    {
        'reviews':['karan write comment','comment like by karan','karan comment like'],
        'output':[1,0,1]
    }
)
data

,reviews,output
0,karan write comment,1
1,comment like by karan,0
2,karan comment like,1


In [3]:
from sklearn.preprocessing import OneHotEncoder
data['reviews'] = data['reviews'].astype('category')
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
ohe_data = ohe.fit_transform(data[['reviews']])
ohe_data


,reviews_comment like by karan,reviews_karan comment like,reviews_karan write comment
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0


In [4]:
# BAG OF WORDS

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
bow = cv.fit_transform(data['reviews'])
cv.vocabulary_

{'karan': 2, 'write': 4, 'comment': 1, 'like': 3, 'by': 0}

In [5]:
print(bow[0].toarray())
print(bow[1].toarray())

[[0 1 1 0 1]]
[[1 1 1 1 0]]


In [6]:
# N-Grams

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(2,2))
n_grams = cv.fit_transform(data['reviews'])
print(cv.vocabulary_)
print(n_grams[0].toarray())
print(n_grams[1].toarray())

{'karan write': 3, 'write comment': 5, 'comment like': 1, 'like by': 4, 'by karan': 0, 'karan comment': 2}
[[0 0 0 1 0 1]]
[[1 1 0 0 1 0]]


In [7]:
# TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_transform = tfidf.fit_transform(data['reviews']).toarray()
print(tfidf_transform)

[[0.         0.45329466 0.45329466 0.         0.76749457]
 [0.66283998 0.39148397 0.39148397 0.50410689 0.        ]
 [0.         0.52284231 0.52284231 0.67325467 0.        ]]


In [8]:
print(tfidf.idf_)
print(tfidf.get_feature_names_out())

[1.69314718 1.         1.         1.28768207 1.69314718]
['by' 'comment' 'karan' 'like' 'write']


In [9]:
## Word2Wec
import pandas as pd
import numpy as np


In [10]:
import gensim
import os

In [11]:
import os

folder_path = '/kaggle/input/game-of-thrones-books'
files = os.listdir(folder_path)
print(files)

['A Storm Of Swords(P3).txt', 'A Feast For Crows(P4).txt', 'A Game Of Thrones(P1).txt', 'A Dance With Dragons(P5).txt', 'A Clash of Kings(P2).txt']


In [25]:
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
import os

story = []

for filename in files:
    f = open(os.path.join(folder_path, filename), 'r', encoding='utf-8', errors='ignore')  # Ensure encoding is handled
    corpus = f.read()
    f.close()  # Properly close the file after reading
    raw_sent = sent_tokenize(corpus)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))


In [37]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)
model.build_vocab(story)
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)


(6574950, 8625240)

In [38]:
model.wv.most_similar('daenerys')

[('stormborn', 0.824991762638092),
 ('unburnt', 0.7632725834846497),
 ('targaryen', 0.7195768356323242),
 ('princess', 0.7085160613059998),
 ('queen', 0.6949372887611389),
 ('rhaegar', 0.6807826161384583),
 ('myrcella', 0.6571646928787231),
 ('viserys', 0.6531112194061279),
 ('khaleesi', 0.6489921808242798),
 ('elia', 0.6473352909088135)]

In [39]:
model.wv.doesnt_match(['jon','rikon','robb','arya','sansa','bran'])

'jon'

In [40]:

model.wv.doesnt_match(['cersei', 'jaime', 'bronn', 'tyrion'])

'bronn'

In [43]:
model.wv.similarity('arya','sansa')

0.8611496

In [44]:
model.wv.similarity('cersei','sansa')

0.7717861

In [45]:
model.wv.similarity('tywin','sansa')

0.16293028

In [46]:
model.wv.get_normed_vectors()

array([[ 6.9494441e-02, -8.2618203e-03, -2.2769666e-03, ...,
        -2.5084770e-01, -4.8343334e-03,  5.8213722e-02],
       [ 2.0241173e-02, -2.4992952e-02,  2.8392393e-02, ...,
        -1.6854145e-02, -6.6411369e-02,  1.2439023e-01],
       [-3.1547327e-02, -9.6925765e-02, -6.4487447e-04, ...,
         5.3476699e-02,  1.1556600e-01, -1.1421640e-01],
       ...,
       [-1.1160887e-01,  8.0429345e-02,  8.5636161e-02, ...,
        -6.4667030e-03,  1.4035197e-01, -1.7283887e-01],
       [ 2.4396402e-04,  4.0501326e-02, -3.2457482e-02, ...,
        -1.2054950e-01,  5.8277347e-03, -1.4471500e-01],
       [ 1.5121881e-02,  5.6570638e-02,  5.5729814e-02, ...,
         3.7496708e-02, -1.4169692e-01, -8.5656188e-02]], dtype=float32)

In [48]:
y = model.wv.index_to_key

In [49]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X = pca.fit_transform(model.wv.get_normed_vectors())
X.shape

(17869, 3)

In [51]:
import plotly.express as px
fig = px.scatter_3d(X[200:300],x=0,y=1,z=2, color=y[200:300])
fig.show()